Data Loading

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense

# Step 1: Data Preprocessing
# Load the two CSV files

data = pd.read_csv("song_artist.csv")

Data Preprocessing

In [2]:
def adjust_weights(activity):
    # Define base weights
    base_weights = {
        'acousticness': 0.1,
        'danceability': 0.2,
        'energy': 0.3,
        'instrumentalness': 0.1,
        'liveness': 0.1,
        'speechiness': 0.2
    }

    # Define weight adjustments based on activity
    activity_adjustments = {
        'gym': {'danceability': 0.2, 'energy': 0.4},
        'relaxation': {'acousticness': 0.3, 'instrumentalness': 0.3},
        'focus': {'instrumentalness': 0.4, 'speechiness': 0.3},
        'party': {'danceability': 0.4, 'energy': 0.5, 'liveness': 0.3}
        # Add more activity adjustments as needed
    }

    # Apply adjustments
    adjusted_weights = base_weights.copy()
    if activity in activity_adjustments:
        for feature, adjustment in activity_adjustments[activity].items():
            adjusted_weights[feature] += adjustment

    return adjusted_weights

In [3]:
# Get user's current activity
user_activity = input("Enter your current activity (gym, relaxation, focus, party, etc.): ").lower()

# Adjust weights based on the user's activity
adjusted_weights = adjust_weights(user_activity)

# Calculate popularity score using adjusted weights
data['popularity'] = sum(data[feature] * weight for feature, weight in adjusted_weights.items())

# Normalize the popularity score to a range between 0 and 100 (optional)
data['popularity'] = (data['popularity'] - data['popularity'].min()) / (data['popularity'].max() - data['popularity'].min()) * 100

In [4]:
data.dtypes

id                           int64
name                        object
artists                     object
acousticness               float64
danceability               float64
energy                     float64
instrumentalness           float64
liveness                   float64
speechiness                float64
acousticness_artist        float64
danceability_artist        float64
energy_artist              float64
instrumentalness_artist    float64
liveness_artist            float64
speechiness_artist         float64
popularity                 float64
dtype: object

In [5]:
data.head()

,id,name,artists,acousticness,danceability,energy,instrumentalness,liveness,speechiness,acousticness_artist,danceability_artist,energy_artist,instrumentalness_artist,liveness_artist,speechiness_artist,popularity
0,0,death bed (coffee for your head) (feat. beabad...,Powfu,0.7310,0.726,0.431,0.00000,0.696,0.1350,0.393250,0.809500,0.408750,0.000062,0.234088,0.131225,60.628731
1,1,THE SCOTTS,THE SCOTTS,0.2330,0.716,0.537,0.00000,0.157,0.0514,0.307500,0.706500,0.521000,0.000000,0.159000,0.052050,56.631914
2,2,The Box,Roddy Ricch,0.1040,0.896,0.586,0.00000,0.790,0.0559,0.184209,0.784984,0.627953,0.000013,0.167566,0.229681,69.174692
3,3,ily (i love you baby) (feat. Emilee),Surf Mesa,0.0686,0.674,0.774,0.00188,0.393,0.0892,0.043200,0.680667,0.709333,0.000627,0.193000,0.058133,69.684680
4,4,Supalonely,BENEE,0.3050,0.863,0.631,0.00003,0.123,0.0534,0.345733,0.757444,0.574333,0.000010,0.145644,0.041956,66.882852


Data Split for training and testing

In [6]:
y = data['popularity']
X = data.drop(columns=['name', 'artists', 'id', 'popularity'])


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Model Training

In [7]:

# Step 2: Model Training
# Define and train a deep learning model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))  # Linear activation for regression task
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test))

Epoch 1/10
2176/2176 [==============================] - 5s 2ms/step - loss: 215.4644 - val_loss: 3.8481
Epoch 2/10
2176/2176 [==============================] - 4s 2ms/step - loss: 0.9040 - val_loss: 0.1792
Epoch 3/10
2176/2176 [==============================] - 4s 2ms/step - loss: 0.1008 - val_loss: 0.0732
Epoch 4/10
2176/2176 [==============================] - 6s 3ms/step - loss: 0.0498 - val_loss: 0.0471
Epoch 5/10
2176/2176 [==============================] - 7s 3ms/step - loss: 0.0324 - val_loss: 0.0190
Epoch 6/10
2176/2176 [==============================] - 6s 3ms/step - loss: 0.0238 - val_loss: 0.0183
Epoch 7/10
2176/2176 [==============================] - 6s 3ms/step - loss: 0.0199 - val_loss: 0.0129
Epoch 8/10
2176/2176 [==============================] - 6s 3ms/step - loss: 0.0173 - val_loss: 0.0102
Epoch 9/10
2176/2176 [==============================] - 6s 3ms/step - loss: 0.0120 - val_loss: 0.0244
Epoch 10/10
2176/2176 [==============================] - 6s 3ms/step - loss: 0.0

Display 5 songs and get user feedback and Reiterate

In [8]:
# Define a blending function to adjust the popularity values
def adjust_popularity(original_popularity, user_rating, user_scale_portion):
    # Define the weights for blending
    original_weight = 0.8  # Weight for the original popularity value
    user_weight = 0.2  # Weight for the user rating and scale portion
    
    # Calculate the adjusted popularity value using a weighted average
    adjusted_popularity = (original_weight * original_popularity) + (user_weight * (user_rating * (user_scale_portion / 10)))
    
    return adjusted_popularity

In [9]:
prompt = 'yes'

while prompt=='yes':

    # SORT THE DATAFRAME AND GET TOP 5 VALUES
    sorted_data = data.sort_values(by='popularity', ascending=False)
    top_songs = sorted_data.head(5)
    top_songs
    print("Songs Recommended for you:")
    for index, song in top_songs.iterrows():
        print(f"{song['name']} by {song['artists']}")

    #GET USER FEEDBACK ON THE 5 SONGS
    user_feedback = {}
    for i, song in top_songs.iterrows():
        feedback = input(f"On a scale of 1 to 10, how much do you like '{song['name']}' by {song['artists']}? ")
        user_feedback[song['name']]=(float(feedback))
    

    #UPDATE THE MAIN DATASET BASE ON USER RECOMMENDATIONS
    for song_name, rating in user_feedback.items():
        rows = data[data['name'] == song_name]
        if not rows.empty:
            original_popularity = rows['popularity'].iloc[0]  
            adjusted_popularity = adjust_popularity(original_popularity, rating, user_scale_portion=10)  
            data.loc[data['name'] == song_name, 'popularity'] = adjusted_popularity

    print("Do you want to continue: (yes/no) Enter no to exit\n")
    prompt = input()
print("\nThank You for using out System :)")



    

Songs Recommended for you:
Pissed Off by Dane Cook
I Didn't Come From a Monkey, I Came From Jesus/Evolution by Joe Rogan
The Improv Fairy Tale by Mitch Hedberg
Free Dog by Bill Burr
The Worse Word by John Mulaney
Do you want to continue: (yes/no) Enter no to exit


Thank You for using out System :)


In [3]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth


In [4]:
# Replace these with your app's credentials
CLIENT_ID = '3721eeb82ff64e7fa6160c2ffb56eb3e'
CLIENT_SECRET = 'eb08b2338acd4ceaa43764039288df53'
REDIRECT_URI = 'http://localhost:8888/callback'

# Scope for playback control
SCOPE = 'user-modify-playback-state user-read-playback-state'

# Authenticate and get the access token
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                               client_secret=CLIENT_SECRET,
                                               redirect_uri=REDIRECT_URI,
                                               scope=SCOPE))

In [5]:

def play_song_on_pc(song_name, device_name=''):
    # Search for the song
    results = sp.search(q=song_name, limit=1, type='track')
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        track_id = track['id']
        track_name = track['name']
        artist_name = track['artists'][0]['name']

        print(f"Playing {track_name} by {artist_name}")

        # Get the user's devices
        devices = sp.devices()
        device_id = None
        for device in devices['devices']:
            # Find the device with the specified name or default to PC-type device
            if device['name'] == device_name or device['type'] == 'Computer':
                device_id = device['id']
                break

        if device_id:
            # Start playback on the device
            sp.start_playback(device_id=device_id, uris=[f'spotify:track:{track_id}'])
        else:
            print(f"Device '{device_name}' not found.")
    else:
        print("Track not found.")

# Example usage: replace 'Your Song Name' with the actual song name you want to play
play_song_on_pc('Your Song Name')


Playing Lord I Lift Your Name On High by Gospel Souls


HTTP Error for PUT to https://api.spotify.com/v1/me/player/play?device_id=eb5906f667985a4bd4988b0d05520446c9e1d30c with Params: {} returned 403 due to Player command failed: Premium required


SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/me/player/play?device_id=eb5906f667985a4bd4988b0d05520446c9e1d30c:
 Player command failed: Premium required, reason: PREMIUM_REQUIRED